## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

In [ ]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from lazy_object_proxy.utils import await_
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [ ]:
load_dotenv(override=True)

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use free and low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern. Also student Christian W. pointed out that OpenAI can sometimes charge for multiple searches for a single call, so it could sometimes cost more than 2.5 cents per call.

Costs are here: https://platform.openai.com/docs/pricing#web-search

In [ ]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search Agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-5-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [ ]:
message = "Latest AI Agent framework in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)


display(Markdown(result.final_output))

### As always, take a look at the trace

https://platform.openai.com/traces

### We will now use Structured Outputs, and include a description of the fields

In [ ]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

In [ ]:
# Use Pydentic to define the Schema of our response - this is know as "Structured Outputs"

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="")



planner_agent = Agent(
    name="Planner Agent",
    instructions=INSTRUCTIONS,
    model="gpt-5-mini",
    output_type=WebSearchItem,
    model_settings=ModelSettings(tool_choice="required"),
)

In [ ]:
message = "Latest AI Agent framework in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

In [ ]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """Send out an email with the given subject and HTML body. """
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("")
    to_email = Email("")
    content = Content("text/html", html_body)
    mail = Mail(from_email, subject, to_email, content)
    response = sg.client.mail.send.post(request_body=mail.get())
    return {"status": "success"}

In [ ]:
send_email

In [4]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email Agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-5-mini",
)

In [ ]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

class ReportData(BaseModel):
    short_summary: str = Field(description="")

    markdown_report: str = Field(description="")

    follow_up_questions: list[str] = Field(description="")


writer_agent = Agent(
    name="Writer Agent",
    instructions=INSTRUCTIONS,
    model="gpt-5-mini",
    output_type=ReportData,
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [ ]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query"""
    print("Planning searches .....")
    result = await Runner.run(planner_agent, f"{query}")
    print(f"Will perform {len(result.final_output)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search agent """
    print("Searching.......")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """Use the Search agent to run a web search for each item in the search plan"""
    input = f"Search term: {item.query}\n Reason for Searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and email it

In [ ]:
async def write_report(query: str, search_results: list[str]):
    """Use the writer agent to write a report based on the search results """
    print("Thinking about report...")
    input = f"Original query: {query}\n Summarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing")
    return result.final_output

async def send_email(report: ReportData):
    """Use the email agent to send an email with the report"""
    print("Writing report...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email Sent")
    return report

### Showtime

In [ ]:
query = "Latest AI Agent framework in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)
    print("Finished research")

### As always, take a look at the trace

https://platform.openai.com/traces